<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/ANMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejemplos de ANMI

In [1]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols

import numpy as np
from numpy.linalg import cond as numero_condicion

from scipy.linalg import orth

## Funciones genéricas



In [2]:
def norma(x):
  return sqrt((x.T * x)[0])

def print_verbose(msg, verbose):
  if type(msg) == str:
    msg = [msg]
  if verbose:
    for i in msg:
      print(i)

## Tema 2

### Número de condicion

El número de condición se define como $\vert\vert A \vert\vert \cdot \vert\vert A^{-1}\vert\vert$. Un número de condición cercano a 1 implica una matriz más estable frente a métodos con elementos diferenciales. Se cumple que las matrices ortogonales tienen número de condición 1.

In [3]:
M = mat(((1, 2, 3), (2, 3, 1), (3, 2, 4)))
M

Matrix([
[1, 2, 3],
[2, 3, 1],
[3, 2, 4]])

In [4]:
numero_condicion(np.array(M).astype(int))

6.960250455451664

In [5]:
M_ort = (orth(np.array(M).astype(int)))
M_ort

array([[-0.50127996, -0.23313726,  0.83328592],
       [-0.45615124,  0.88953592, -0.02553206],
       [-0.73528528, -0.39290311, -0.55225239]])

In [6]:
numero_condicion(M_ort)

1.0000000000000004

### Factorización LU y LDU*

Recordemos que para una matriz, la factorización LU es el proceso de aplicación de la simplificación de Gauss, de modo que la matriz $L$ es una matriz triangular inferior con los coeficientes de transformación, y la matriz $U$ es la matriz superior con los elementos tras las transformaciones lineales.

Además, se puede hacer que $D$ sea una matriz diagonal con los valores de la diagonal de $U$, de modo que $LU$ = $LDD^{-1}U$, y si hacemos $U^* = D^{-1}U$ entonces tenemos $LDU^*$, donde $U^*$ sigue siendo una matriz diagonal superior, pero con la diagonal igual a 1.

A la hora de aplicar la factorización LU y LDU* se puede hacer una permutación de filas, de modo que en cada iteración se coge la fila con mayor valor (de entre las que no se han *procesado*) y se permuta, garantizando una solución siempre. También, es importante tener en mente que la factorización falla si algún elemento de la diagonal (desde el principio o durante la factorización) es 0, de modo que para solucionar ese caso se aplica la permutación.

Todas las permutaciones quedan recogidas en una matriz $P$, de modo que $$LU = LDU^* = PA$$

In [7]:
# Funciones asociadas a la factorización LU y LUD

def permutacion_matriz(U, fila_i, idx_max, verbose=False, P=None, r=None):
      print_verbose([f'Permutamos fila {fila_i} con {idx_max}',
                    f'U antes:\n {np.array(U)}',
                    f'P antes:\n {np.array(P)}'], verbose)

      if fila_i != idx_max:
        fila_origen, fila_destino = U[fila_i, :].copy(), U[idx_max, :].copy()
        U[idx_max, :], U[fila_i, :] = fila_origen, fila_destino
        if P is not None:
          fila_origen, fila_destino = P[fila_i, :].copy(), P[idx_max, :].copy()
          P[idx_max, :], P[fila_i, :] = fila_origen, fila_destino
        if r is not None:
          fila_origen, fila_destino = r[fila_i, :].copy(), r[idx_max, :].copy()
          r[idx_max, :], r[fila_i, :] = fila_origen, fila_destino

      print_verbose([f'U despues:\n {np.array(U)}',
                    f'P despues:\n {np.array(P)}'], verbose)

      return U, P, r


def permutacion_L(L, perm, verbose=False):
  """
  Esta función la creo porque a la hora de hacer la permutación, hay que 
  permutar los elementos de L pero no directamente. Solo hay que seleccionar
  los elementos de la diagonal inferior que correspondan con el menor número de
  columnas. Por ejemplo, si permutamos las filas 3 y 5, se tienen que mover solo
  los elementos de las 2 primeras columnas (2 -> 3) para luego continuar 
  con las transformaciones del resto de columnas.
  """
  print_verbose(f'L antes:\n {np.array(L)}', verbose)

  fila_origen, fila_destino = L[perm[0], :min(perm)].copy(), L[perm[1], :min(perm)].copy()
  L[perm[1], :min(perm)], L[perm[0], :min(perm)] = fila_origen, fila_destino

  print_verbose(f'L despues:\n {np.array(L)}', verbose)

  return L 


def descomposicion_LU(m, rhs=None, verbose=True, permutar_max=False):
  '''
  Esta función realiza el algoritmo de triangulación de Gauss. Para ello vamos a
  ir aplicando paso a paso el algoritmo tal cual se hace manualmente, 
  y aplicamos los cambios de columnas necesarios si hay que aplicar permutaciones.
  Por defecto, si encontramos un 0 en la diagonal aplicamos permutar_max para esa
  fila, y devolvemos la matriz de permutaciones.
  '''

  if rhs is None:
    rhs = zeros(m.shape[0], 1)

  print_verbose(["La matriz M|X es  (X = 0) si no se ha introducido", 
                np.concatenate((np.array(m), np.array(rhs)), axis=1)], verbose)
  
  P, L, U, r = eye(m.shape[0]), eye(m.shape[0]), m.copy(), rhs.copy()
  lista_perms = []
  fila_i, err, err_max = 0, 0, 3
  

  while fila_i < m.shape[0] and err < err_max:
    print_verbose([f'\n=====================================\nFila {fila_i}', 
                   f'A {fila_i}, {err}'], verbose)

    if U[max(0, fila_i-1), max(0, fila_i-1)] == 0 or permutar_max:
      """
      Esta parte nos asegura que si un elemento diagonal es cero, permutamos
      la fila con su inmediata inferior y rehacemos los cálculos, 
      y así tener una configuración viable.
      En cualquier permutación tenemos que cambiar la L acorde con el cambio.
      Al hacer la permutación, con los nuevos valores, repasamos la matriz para 
      asegurarnos de que todos los puntos están cumplidos.

      Para la opción de permutar, buscamos un elemento de las filas no alteradas
      que sea el mayor. Si hay más de una fila, y una de ellas es la 
      actual, no aplica la permutación.
      """
      sub_mat = U[fila_i:, fila_i:]
      max_el = np.max(np.array(sub_mat))
      idx_max = np.min([i[0] for i in np.argwhere(np.array(sub_mat) == max_el)]) + fila_i # Cogemos el primer elemento si hay varios
      
      if U[max(0, fila_i-1), max(0, fila_i-1)] == 0:
        U, P, r = permutacion_matriz(U, max(0, fila_i-1), idx_max, verbose, P, r)
        L = permutacion_L(L, [max(0, fila_i-1), idx_max], verbose)
        fila_i = min(max(0, fila_i-1), idx_max)
        
        err += 1
        print_verbose(f'Err {err}', verbose)
        continue

      elif permutar_max:
        if idx_max <= fila_i: # No es necesario hacer la permutación
          print_verbose('El índice de permutación es igual a la fila a permutar.', verbose)
        else:
          U, P, r = permutacion_matriz(U, fila_i, idx_max, verbose, P, r)
          L = permutacion_L(L, [fila_i, idx_max], verbose)
          continue

    # Ahora aplicamos el algoritmo de calculo de filas:
    for columna_j in range(fila_i):
      a_ij = U[fila_i, columna_j] /  U[columna_j, columna_j]
      
      if a_ij != 0:
        L[fila_i, columna_j] = a_ij
        U[fila_i, :] = U[fila_i, :] - a_ij * U[columna_j, :]
        r[fila_i, :] = r[fila_i, :] - a_ij * r[columna_j, :]

      err = 0

      print_verbose([
        f'||||||||||||||||||||||||\n Columna {columna_j}',
        f'a_{fila_i},{columna_j} = {a_ij}',
        f'P = \n{np.array(P)}',
        f'L = \n{np.array(L)}',
        f'U = \n{np.array(U)}',
        f'r = \n{np.array(r)}'], verbose)

    fila_i += 1
  
  if err == err_max:
    print('Algo ha ido mal... mira el log.')

  print_verbose([
        f'\/\/\/\/\/\/\/\/\/ FORMA FINAL',
        f'a_{fila_i},{columna_j} = {a_ij}',
        f'P = \n{np.array(P)}\n',
        f'L = \n{np.array(L)}\n',
        f'U = \n{np.array(U)}\n',
        f'r = \n{np.array(r)}\n'], verbose)

  if simplify(L * U) != simplify(P * m):
    print('AVISO!!! LU != PA')

  return {'P': simplify(P), 'L': simplify(L), 'U': simplify(U), 'r': simplify(r)}


def descomposicion_LDU(m, permutar_max=True, verbose=False):
  dict_LU = descomposicion_LU(m, permutar_max=permutar_max, verbose=verbose)
  L, U = dict_LU['L'], dict_LU['U']

  D = zeros(m.shape[0], m.shape[0])

  for i in range(U.shape[0]):
    D[i, i] = U[i, i]
  
  U = D.inv() * U

  print_verbose([
        f'\/\/\/\/\/\/\/\/\/ RESULTADOS LDU*',
        f'L = \n{np.array(L)}\n',
        f'D = \n{np.array(D)}\n',
        f'U* = \n{np.array(U)}\n'], verbose)

  if simplify(L * D * U) != simplify(dict_LU['P'] * m):
    print('AVISO!!! LDU != PA')

  return {'P': simplify(dict_LU['P']), 'L': simplify(L), 'U': simplify(U), 'D': simplify(D)}
  
  


In [8]:
M = mat(((1, 4, 4), (3, 2, 1), (2, 4, 1)))

In [9]:
descomposicion_LU(M, permutar_max=False)

La matriz M|X es  (X = 0) si no se ha introducido
[[1 4 4 0]
 [3 2 1 0]
 [2 4 1 0]]

Fila 0
A 0, 0

Fila 1
A 1, 0
||||||||||||||||||||||||
 Columna 0
a_1,0 = 3
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
L = 
[[1 0 0]
 [3 1 0]
 [0 0 1]]
U = 
[[1 4 4]
 [0 -10 -11]
 [2 4 1]]
r = 
[[0]
 [0]
 [0]]

Fila 2
A 2, 0
||||||||||||||||||||||||
 Columna 0
a_2,0 = 2
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
L = 
[[1 0 0]
 [3 1 0]
 [2 0 1]]
U = 
[[1 4 4]
 [0 -10 -11]
 [0 -4 -7]]
r = 
[[0]
 [0]
 [0]]
||||||||||||||||||||||||
 Columna 1
a_2,1 = 2/5
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]
L = 
[[1 0 0]
 [3 1 0]
 [2 2/5 1]]
U = 
[[1 4 4]
 [0 -10 -11]
 [0 0 -13/5]]
r = 
[[0]
 [0]
 [0]]
\/\/\/\/\/\/\/\/\/ FORMA FINAL
a_3,1 = 2/5
P = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]

L = 
[[1 0 0]
 [3 1 0]
 [2 2/5 1]]

U = 
[[1 4 4]
 [0 -10 -11]
 [0 0 -13/5]]

r = 
[[0]
 [0]
 [0]]



{'L': Matrix([
 [1,   0, 0],
 [3,   1, 0],
 [2, 2/5, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]), 'U': Matrix([
 [1,   4,     4],
 [0, -10,   -11],
 [0,   0, -13/5]]), 'r': Matrix([
 [0],
 [0],
 [0]])}

In [10]:
descomposicion_LDU(M, permutar_max=False)

{'D': Matrix([
 [1,   0,     0],
 [0, -10,     0],
 [0,   0, -13/5]]), 'L': Matrix([
 [1,   0, 0],
 [3,   1, 0],
 [2, 2/5, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]), 'U': Matrix([
 [1, 4,     4],
 [0, 1, 11/10],
 [0, 0,     1]])}

In [11]:
descomposicion_LU(M, rhs=ones(M.shape[0], 1), permutar_max=True)

La matriz M|X es  (X = 0) si no se ha introducido
[[1 4 4 1]
 [3 2 1 1]
 [2 4 1 1]]

Fila 0
A 0, 0
El índice de permutación es igual a la fila a permutar.

Fila 1
A 1, 0
Permutamos fila 1 con 2
U antes:
 [[1 4 4]
 [3 2 1]
 [2 4 1]]
P antes:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
U despues:
 [[1 4 4]
 [2 4 1]
 [3 2 1]]
P despues:
 [[1 0 0]
 [0 0 1]
 [0 1 0]]
L antes:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
L despues:
 [[1 0 0]
 [0 1 0]
 [0 0 1]]

Fila 1
A 1, 0
El índice de permutación es igual a la fila a permutar.
||||||||||||||||||||||||
 Columna 0
a_1,0 = 2
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[[1 0 0]
 [2 1 0]
 [0 0 1]]
U = 
[[1 4 4]
 [0 -4 -7]
 [3 2 1]]
r = 
[[1]
 [-1]
 [1]]

Fila 2
A 2, 0
El índice de permutación es igual a la fila a permutar.
||||||||||||||||||||||||
 Columna 0
a_2,0 = 3
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[[1 0 0]
 [2 1 0]
 [3 0 1]]
U = 
[[1 4 4]
 [0 -4 -7]
 [0 -10 -11]]
r = 
[[1]
 [-1]
 [-2]]
||||||||||||||||||||||||
 Columna 1
a_2,1 = 5/2
P = 
[[1 0 0]
 [0 0 1]
 [0 1 0]]
L = 
[

{'L': Matrix([
 [1,   0, 0],
 [2,   1, 0],
 [3, 5/2, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0]]), 'U': Matrix([
 [1,  4,    4],
 [0, -4,   -7],
 [0,  0, 13/2]]), 'r': Matrix([
 [  1],
 [ -1],
 [1/2]])}

In [12]:
descomposicion_LDU(M, permutar_max=True)

{'D': Matrix([
 [1,  0,    0],
 [0, -4,    0],
 [0,  0, 13/2]]), 'L': Matrix([
 [1,   0, 0],
 [2,   1, 0],
 [3, 5/2, 1]]), 'P': Matrix([
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0]]), 'U': Matrix([
 [1, 4,   4],
 [0, 1, 7/4],
 [0, 0,   1]])}

### Factorización de Cholesky

La factorización de Cholesky es una factorización que genera una matriz triangular inferior $L$ tal que $A = LL^T$. Para que una matriz sea factorizable, tiene que cumplir que sus menores principales sean positivos, y que sea simétrica.

In [13]:
def cholesky(m, verbose=False):
  """
  Primero comprobamos que los menores sean positivos. Eso es equivalente que
  sus autovalores sean positivos. Por simplificar, si el determinante es negativo
  ya descartamos que sea factorizable, y saltamos el warning.
  """

  if m != m.T:
    print('AVISO! La matriz no es simétrica, y por tanto no factorizable por Cholesky.')

  print_verbose(f"|M| es {m.det()}. Si es < 0, no es factorizable.", verbose)

  m_chol = zeros(m.shape[0], m.shape[1])

  for col_j in range(m.shape[0]):
    for row_i in range(col_j, m.shape[0]):
      if col_j == 0:
        if row_i == 0:
          m_chol[row_i, col_j] = sqrt(m[row_i, col_j])
        else:
          m_chol[row_i, col_j] = m[row_i, col_j] / m_chol[0, 0]
      
      else:
        if col_j == row_i:
          m_chol[row_i, col_j] = sqrt(m[col_j, col_j] - sum([m_chol[col_j, k] 
                                                      ** 2 for k in range(col_j)]))
        else:
          m_chol[row_i, col_j] = (m[row_i, col_j] - sum([m_chol[col_j, k] * m_chol[row_i, k] for k in range(col_j)]))/(m_chol[col_j, col_j])

  if simplify(m_chol * m_chol.T) != m:
    print(f"AVISO!!! La matriz no es Cholesky-zable. \n L = \n {np.array(m_chol)} \n\n L*L.T = \n {np.array(m_chol * m_chol.T)}")

  return simplify(m_chol)

In [14]:
M = mat(((2, -1, 0), (-1, 2, -2), (0, 2, 1)))
M

Matrix([
[ 2, -1,  0],
[-1,  2, -2],
[ 0,  2,  1]])

In [15]:
cholesky(M)

AVISO! La matriz no es simétrica, y por tanto no factorizable por Cholesky.
AVISO!!! La matriz no es Cholesky-zable. 
 L = 
 [[sqrt(2) 0 0]
 [-sqrt(2)/2 sqrt(6)/2 0]
 [0 2*sqrt(6)/3 sqrt(15)*I/3]] 

 L*L.T = 
 [[2 -1 0]
 [-1 2 2]
 [0 2 1]]


Matrix([
[   sqrt(2),           0,            0],
[-sqrt(2)/2,   sqrt(6)/2,            0],
[         0, 2*sqrt(6)/3, sqrt(15)*I/3]])

In [16]:
cholesky(M + M.T)

Matrix([
[ 2,       0,       0],
[-1, sqrt(3),       0],
[ 0,       0, sqrt(2)]])

In [17]:
M + M.T

Matrix([
[ 4, -2, 0],
[-2,  4, 0],
[ 0,  0, 2]])

In [18]:
cholesky(M + M.T) * cholesky(M + M.T).T

Matrix([
[ 4, -2, 0],
[-2,  4, 0],
[ 0,  0, 2]])

In [19]:
# Podemos hacer también Cholesky a una matriz con símbolos!
x = symbols('x')

Mx = mat(((1, x, 1), (x, 2, 1), (1, 1, 3)))
Mx

Matrix([
[1, x, 1],
[x, 2, 1],
[1, 1, 3]])

In [20]:
cholesky(Mx)

Matrix([
[1,                      0,                                    0],
[x,         sqrt(2 - x**2),                                    0],
[1, (1 - x)/sqrt(2 - x**2), sqrt((-3*x**2 + 2*x + 3)/(2 - x**2))]])

### Ortogonalización de Gram-Schmidt




In [21]:
def suma_columnas(lista):
  if len(lista) > 0:
    m = zeros(lista[0].shape[0], 1)
    for i in lista:
      m += i
    return m
  else:
    return 0

def gram_schmidt(m, verbose=False):
  """
  La ortogonalización produce una matriz ortogonalizada por columnas.
  p es la matriz ortogonal y p_norm es la ortonormal
  c es la matriz triangular tal que cij = aj·pi/||pi||^2 (los coeficientes de ortogonalización). Estos coeficientes se usan para la factorización QR.
  """

  p = zeros(m.shape[0], m.shape[1])

  p[:, 0] = m[:, 0]

  for col in range(1, m.shape[1]):
    p[:, col] = m[:, col] - suma_columnas([(m[:, col].T * p[:, i])[0]/(p[:, i].T * p[:, i])[0] * p[:, i] for i in range(0, col)])

  print_verbose(f"La matriz ortogonal es \n {p}", verbose)


  p_norm = zeros(m.shape[0], m.shape[1])
  for col in range(p.shape[1]):
    p_norm[:, col] = p[:, col] / (p[:, col].T * p[:, col])[0]
  
  print_verbose(f"La matriz ortonormal es \n {p_norm}", verbose)


  c = zeros(m.shape[0], m.shape[1])
  for col in range(1, m.shape[1]):
    for row in range(0, col):
      c[row, col] = (m[:, col].T * p[:, row])[0]/(p[:, row].T * p[:, row])[0]

  return {'P': simplify(p), 'Pn': simplify(p_norm), 'c': simplify(c)} 


In [22]:
M

Matrix([
[ 2, -1,  0],
[-1,  2, -2],
[ 0,  2,  1]])

In [23]:
GS = gram_schmidt(M)
GS['P']

Matrix([
[ 2, 3/5, -22/29],
[-1, 6/5, -44/29],
[ 0,   2,  33/29]])

In [24]:
GS['Pn']

Matrix([
[ 2/5,  3/29, -2/11],
[-1/5,  6/29, -4/11],
[   0, 10/29,  3/11]])

In [25]:
GS['c']

Matrix([
[0, -4/5,   2/5],
[0,    0, -2/29],
[0,    0,     0]])

In [26]:
GS['P'][:, 1].T * GS['P'][:, 2]

Matrix([[0]])

In [27]:
Mx = mat(((1, 2, 3), (1, x, 1), (0, 0, 3)))

In [28]:
GSx = gram_schmidt(Mx)
GSx['P']

Matrix([
[1, 1 - x/2, 0],
[1, x/2 - 1, 0],
[0,       0, 3]])

### Transformación de Householder

La transformación de Householder es una transformación para pasar de un vertor $x$ a un vector $y$. Para ello se toma el vector $e$, que sería el eje de transformación, y la matriz aplicación es $H = I - 2ee^t$.

En este caso, $$e = \pm \frac{x - y}{\vert\vert x - y \vert\vert}$$

Para la transformación de Householder $x$ e $y$ **tienen que tener la misma norma**. El vector resultante $e$ tiene norma 1.


In [29]:
def householder(x, y, signo='+', normalizar = False):
  """
  Aplica la transformación de Householder. x e y pueden ser vectores fila o columna, 
  en cuyo caso aplicamos la transformación de una forma u otra.
  """
  if signo == '+':
    mult = 1
  else:
    mult = -1

  if normalizar:
    x = simplify(x / norma(x))
    y = simplify(y / norma (y))

  if simplify(norma(x)) != simplify(norma(y)):
    print(f'AVISO!!! x tiene norma {norma(x)} e y tiene norma {norma(y)}. Para Householder las normas tienen que ser iguales.')

  if simplify(norma(x - y)) == 0:  # x==y
    e = zeros(x.shape[0], x.shape[1])
  else:
    e = mult * (x - y)/(norma(x - y))

  if (e.shape[0] == 1) and (e.shape[1] >= 1): # El vector es fila:
    H = eye(x.shape[1]) - 2 * e.T * e
  else:
    H = eye(x.shape[0]) - 2 * e * e.T
  

  if (e.shape[0] == 1) and (e.shape[1] >= 1) and (simplify(x * H) != y):
    print(f'AVISO FILAS!!! xH != y. Para Householder las normas tienen que ser iguales.')
  elif (e.shape[1] == 1) and (e.shape[0] >= 1) and (simplify(H * x) != y):
    print(f'AVISO!!! Hx != y. Para Householder las normas tienen que ser iguales.')

  return simplify(H), simplify(e)

In [30]:
v0 = mat(((1, 0)))
vf = mat((1, 1))
vf = vf/norma(vf)

In [31]:
H, e = householder(v0, vf, normalizar=False)
H

Matrix([
[sqrt(2)/2,  sqrt(2)/2],
[sqrt(2)/2, -sqrt(2)/2]])

In [32]:
e

Matrix([
[           sqrt(2 - sqrt(2))/2],
[-sqrt(2)/(2*sqrt(2 - sqrt(2)))]])

In [33]:
# Ejercicio 10
t = symbols('t')

v0 = mat(((1, 0)))
vf = mat((cos(t), sin(t)))

H, e = householder(v0, vf, normalizar=False)

In [34]:
H

Matrix([
[cos(t),  sin(t)],
[sin(t), -cos(t)]])

In [35]:
e

Matrix([
[      sqrt(2 - 2*cos(t))/2],
[-sin(t)/sqrt(2 - 2*cos(t))]])

### Factorización QR

La factorización QR consiste en transformar $A = QR$ donde $Q$ es ortogonal y $R$ es triangular superior. 

Si $P$ es la matriz ortogonalizada, $C$ es la matriz con los factores de ortonormalización (para Gram-Schmidt, por ejemplo, es $m_{ij} = \frac{a^j\cdot p^i}{\vert\vert p^{i}\vert\vert^2}$ y $D$ es la matriz de las normas de los vectores ortogonales ($\vert\vert p^i\vert\vert$), entonces se tiene que:
$$Q = PD^{-1}$$
$$R = D(I + C)$$

Para el método de Hausholder, vamos a ir por columnas. Para cada columna, seleccionamos la submatriz $N(i) = A_{ii}$ de (n-i) filas y columnas. Para esa matriz, vamos a hacer que la primera columna, $p_i$ pase a ser $\vert\vert p_i\vert\vert(

In [41]:
def factorizacion_QR(m, verbose=True, metodo='gram_schmidt'):
  if metodo == 'gram_schmidt':
    print_verbose('Aplicamos QR con Gram Schmidt', verbose)

    dict_gs = gram_schmidt(m)
    P, C = dict_gs['P'], dict_gs['c']
  
    D = zeros(m.shape[0], m.shape[1])
    for col in range(m.shape[0]):
      D[col, col] = sqrt((P[:, col].T * P[:, col])[0])
    
    Q = simplify(P * (D ** (-1)))
    R = simplify(D * (eye(m.shape[0]) + C))

  elif metodo == 'hausholder':
    print_verbose('Aplicamos QR con Hausholder')
    lista_p = []

    for col in range(m.shape[1]):



      pass

  else:
    raise "El método no existe. Los disponibles son gram_schmidt y hausholder"
  
  print_verbose(f'Q es \n{np.array(Q)}\nR es \n{np.array(R)}\nD es \n{np.array(D)}', verbose)

  if m != Q * R:
    print('AVISO!!! A != QR')

  return {'Q': Q, 'R': R, 'D': D} 

In [42]:
M = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [43]:
factorizacion_QR(M)

Aplicamos QR con Gram Schmidt
Q es 
[[1 0 0]
 [0 0 -1]
 [0 1 0]]
R es 
[[2 -1 0]
 [0 2 -1]
 [0 0 2]]
D es 
[[2 0 0]
 [0 2 0]
 [0 0 2]]


{'D': Matrix([
 [2, 0, 0],
 [0, 2, 0],
 [0, 0, 2]]), 'Q': Matrix([
 [1, 0,  0],
 [0, 0, -1],
 [0, 1,  0]]), 'R': Matrix([
 [2, -1,  0],
 [0,  2, -1],
 [0,  0,  2]])}

In [44]:
A1 = mat(((2, -1, 0), (0, 0, -2), (0, 2, -1)))

In [46]:
A1 = mat(((1, 2, 3), (4, 5, 6), (7, 8, 9)))
A1

Matrix([
[1, 2, 3],
[4, 5, 6],
[7, 8, 9]])

In [47]:
A1

Matrix([
[1, 2, 3],
[4, 5, 6],
[7, 8, 9]])

In [48]:
a_1 = A1[:, 0]
v_1 = zeros(a_1.shape[0], 1)
v_1[0] = norma(a_1)
P1, e1 = householder(a_1, v_1)

In [49]:
P1, e1

(Matrix([
 [  sqrt(66)/66,             2*sqrt(66)/33,             7*sqrt(66)/66],
 [2*sqrt(66)/33,   49/65 - 8*sqrt(66)/2145, -28/65 - 14*sqrt(66)/2145],
 [7*sqrt(66)/66, -28/65 - 14*sqrt(66)/2145,  16/65 - 49*sqrt(66)/4290]]),
 Matrix([
 [sqrt(2)*(1 - sqrt(66))/(2*sqrt(66 - sqrt(66)))],
 [                 2*sqrt(2)/sqrt(66 - sqrt(66))],
 [             7*sqrt(2)/(2*sqrt(66 - sqrt(66)))]]))

In [50]:
norma(a_1)

sqrt(66)